In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# Data Loading
dirty_data = pd.read_csv('../../datasets/initial_datasets/videos.csv')
clean_data = pd.read_csv('../../datasets/cleaned_datasets/cleaned_videos.csv')

# Selection of features variables (columns) and target variable (column)
features = ['views', 'likes', 'dislikes', 'comment_count'] # plus title, tags, description
target = 'category_id'

# Creation of dataframes with features and target
X_dirty = dirty_data[features]
y_dirty = dirty_data[target]

X_clean = clean_data[features]
y_clean = clean_data[target]

# Split data for training and testing
X_dirty_train, X_dirty_test, y_dirty_train, y_dirty_test = train_test_split(X_dirty, y_dirty, test_size=0.2, random_state=42)
X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Model training on dirty data
model_dirty = RandomForestClassifier(random_state=42)
model_dirty.fit(X_dirty_train, y_dirty_train)

# Model training on cleaned data
model_clean = RandomForestClassifier(random_state=42)
model_clean.fit(X_clean_train, y_clean_train)

# Prediction and evaluation of the model
y_dirty_pred = model_dirty.predict(X_dirty_test)
accuracy_dirty = accuracy_score(y_dirty_test, y_dirty_pred)
f1_dirty = f1_score(y_dirty_test, y_dirty_pred, average='weighted')

# Predizione e valutazione del modello pulito
y_clean_pred = model_clean.predict(X_clean_test)
accuracy_clean = accuracy_score(y_clean_test, y_clean_pred)
f1_clean = f1_score(y_clean_test, y_clean_pred, average='weighted')

# Confronto delle prestazioni
print(f'Model trained on dirty data: Accuracy = {accuracy_dirty}, F1 Score = {f1_dirty}')
print(f'Model trained on clean data: Accuracy = {accuracy_clean}, F1 Score = {f1_clean}')


/var/folders/29/hp9758mj5zj7dcbpxts76vt00000gn/T/ipykernel_97803/4028560899.py:2: DtypeWarning: Columns (0,1,2,3,5,6,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dirty_data = pd.read_csv('../../datasets/initial_datasets/videos.csv')


ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values